## Import Libraries

In [1]:
import pandas as pd
import os
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Read Config File

In [2]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
output_folder = config['DEFAULT']['Output-Folder']
db_name = config['DEFAULT']['DB-Name']
user_name = config['DEFAULT']['User-Name']
psword = config['DEFAULT']['Psword']
collection_number = config['DEFAULT']['Collection-Number']
collection_name = config['DEFAULT']['Collection-Name']

## Connect MOngoDB

In [3]:
from pymongo import MongoClient
client = MongoClient(ip, int(port), username=user_name, password=psword)

## Get Collection Names

In [4]:
db = client[db_name]
collections = db.collection_names()

## Supporting Functions

In [5]:
# create foler if not exist
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
# export csv from MongoDB
def export_csv_mongodb(c):
    print("Processing on : " + c)
    cursor = db[c].find({})
    data = list(cursor)
    
    # get columns
    columns = list(data[0].keys())
    
    # delete '_id'
    if '_id' in columns:
        columns.remove('_id')
        
    try:
        with open(output_folder + c+".csv", 'w') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()
            for i in data:
                del i["_id"]
                writer.writerow(i)
    except IOError:
        print("I/O error") 
        
    print("CSV File for " + c + " is finished.")
    print("------------------------------------")

## Export CSV from MongoDB

In [6]:
create_folder(output_folder)

if collection_number == "all":
    for c in collections:
         export_csv_mongodb(c)
else:
    export_csv_mongodb(collection_name)  